# Integer Linear Programming Using Deep Neural Network
## Model Architecture

Input Layer:
Input size: n (number of decision variables).

First Fully Connected Layer (fc1):
Input size: n.
Output size: 128.
Activation function: ReLU (Rectified Linear Unit).

Second Fully Connected Layer (fc2):
Input size: 128.
Output size: 64.
Activation function: ReLU.

Third Fully Connected Layer (fc3):
Input size: 64.
Output size: n.

No activation function directly applied to the output, but a custom activation function (custom_activation) is used to ensure non-negative outputs.

Initialization
Weight Initialization: Kaiming uniform initialization (He initialization) is used for the weights of all the fully connected layers to ensure proper initialization for ReLU activation functions.

Custom Activation
Custom Activation Function: A custom activation function custom_activation is applied to the final output to ensure all output values are non-negative by clamping them to a minimum value of 0.

In [1]:
import torch
from torch import nn
from torch.optim import Adam

# Set random seed for reproducibility
torch.manual_seed(0)

######################################################################################################################
# # ILP problem for n=2
# c = torch.tensor([3, 2], dtype=torch.float32)  # Objective function coefficients
# A = torch.tensor([[2, 1], [1, 3]], dtype=torch.float32)  # Constraint coefficients matrix
# b = torch.tensor([10, 12], dtype=torch.float32)  # Right-hand side vector

# # ILP problem for n=2
# c = torch.tensor([5, 7], dtype=torch.float32)  # Objective function coefficients
# A = torch.tensor([[3, 2], [2, 3]], dtype=torch.float32)  # Constraint coefficients matrix
# b = torch.tensor([18, 21], dtype=torch.float32)  # Right-hand side vector

# ILP problem (generalized for n variables) (this problem gave accurate result, others were very close to optimal)
n = 5  # Number of variables
c = torch.tensor([5, 7, 3, 4, 6], dtype=torch.float32)  # Objective function coefficients
A = torch.tensor([[3, 2, 1, 4, 0], 
                  [2, 3, 0, 0, 1], 
                  [1, 1, 2, 2, 2]], dtype=torch.float32)  # Constraint coefficients matrix
b = torch.tensor([18, 21, 19], dtype=torch.float32)  # Right-hand side vector

# # ILP problem for n = 6
# n = 6  # Number of variables
# c = torch.tensor([8, 6, 5, 4, 3, 7], dtype=torch.float32)  # Objective function coefficients
# A = torch.tensor([
#     [4, 3, 2, 5, 1, 0],
#     [2, 1, 4, 0, 3, 5],
#     [1, 2, 1, 3, 4, 2]
# ], dtype=torch.float32)  # Constraint coefficients matrix
# b = torch.tensor([20, 15, 25], dtype=torch.float32)  # Right-hand side vector

######################################################################################################################

# Custom activation function to ensure outputs are within constraints
def custom_activation(x):
    return torch.clamp(x, min=0)  # Ensure values are non-negative

# Defining neural network model
class ILPNet(nn.Module):
    def __init__(self, input_size):
        super(ILPNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, input_size)
        nn.init.kaiming_uniform_(self.fc1.weight)
        nn.init.kaiming_uniform_(self.fc2.weight)
        nn.init.kaiming_uniform_(self.fc3.weight)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return custom_activation(x)

# Function to train the model
def train_model(model, optimizer, num_epochs=1000, penalty_weight=10000):
    input_vector = torch.zeros(n)  # Use zero vector as input
    best_loss = float('inf')
    best_output = None

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(input_vector)  # Use fixed input vector
        objective_loss = -torch.sum(c * output)  # Objective function loss (maximize c*x)
        constraint_violation = torch.relu(torch.matmul(A, output) - b)  # Constraint violation
        constraint_loss = penalty_weight * torch.sum(constraint_violation)  # Constraint violation loss with penalty
        loss = objective_loss + constraint_loss
        loss.backward()
        optimizer.step()
        
        # Track the best loss and corresponding output
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_output = output.detach().clone()

        if epoch % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}, Objective Loss: {objective_loss.item()}, Constraint Loss: {constraint_loss.item()}")
            print(f"Output: {output.detach().numpy()}")

    return best_output  # Return the best solution directly

# Feasibility correction step
def correct_solution(solution):
    solution = solution.clone()
    while torch.any(torch.matmul(A, solution) > b):
        for i in range(len(solution)):
            if solution[i] > 0:
                solution[i] -= 1
                if torch.all(torch.matmul(A, solution) <= b):
                    return solution
    return solution

# Train multiple models and select the best solution
def find_best_solution(num_restarts=10, num_epochs=1000, penalty_weight=100):
    best_solution = None
    best_objective = float('-inf')

    for _ in range(num_restarts):
        model = ILPNet(n)
        optimizer = Adam(model.parameters(), lr=0.01)
        solution = train_model(model, optimizer, num_epochs, penalty_weight)
        
        # Round to nearest integers
        rounded_solution = torch.round(solution)
        
        # Correct solution to be feasible
        feasible_solution = correct_solution(rounded_solution)
        
        # Check feasibility
        if torch.all(torch.matmul(A, feasible_solution) <= b):
            objective_value = torch.sum(c * feasible_solution).item()
            if objective_value > best_objective:
                best_objective = objective_value
                best_solution = feasible_solution

    return best_solution

# Find the best solution after multiple restarts
best_solution = find_best_solution()
print("Best Predicted Solution:")
print(best_solution)


Epoch [0/1000], Loss: -3.9088692665100098, Objective Loss: -3.9088692665100098, Constraint Loss: 0.0
Output: [0.31848937 0.2673707  0.14827582 0.         0.        ]
Epoch [100/1000], Loss: -56.013511657714844, Objective Loss: -56.013511657714844, Constraint Loss: 0.0
Output: [0.        3.5978646 0.        1.3015724 4.2703614]
Epoch [200/1000], Loss: -56.45018768310547, Objective Loss: -56.45018768310547, Constraint Loss: 0.0
Output: [0.        3.6524088 0.        1.1663665 4.3696427]
Epoch [300/1000], Loss: -73.05298614501953, Objective Loss: -73.05298614501953, Constraint Loss: 0.0
Output: [0.        5.0027966 0.        1.2981735 5.4734516]
Epoch [400/1000], Loss: -63.004241943359375, Objective Loss: -63.004241943359375, Constraint Loss: 0.0
Output: [0.        4.32612   0.        0.9579606 4.8149266]
Epoch [500/1000], Loss: -63.51261520385742, Objective Loss: -63.51261520385742, Constraint Loss: 0.0
Output: [0.        4.277055  0.        1.0021061 4.927468 ]
Epoch [600/1000], Loss: -